In [2]:
#  🚀 Install libraries 
#! pip install tqdm pandas numpy spacy bokeh 

In [3]:
#  🚀 Load german language model for annotation
#! python -m spacy download de_core_news_sm

In [53]:
# load libraries 
import json
import typing
import requests
from pathlib import Path
from time import time
from collections import OrderedDict, Counter

from tqdm import tqdm
import pandas as pd
import numpy as np
import spacy

from bokeh.io import output_notebook, show
from bokeh.layouts import column
from bokeh.models import CustomJS, TextInput, Div

In [54]:
text_path = Path("data/txt/SNP2719372X-19181015-0-0-0-0.txt")

In [55]:
# read text and print some parts of the text
if text_path.is_file():
    text = text_path.read_text()
    print(f"Textauszug:\n {text[10280:10400]}")
else:
    print("The file path does not exist. Set the variable text_path to an existing path.")

Textauszug:
 onar Law haben London verlassen, um sich mit Ltoyd George zu besprechen. Weiter erfÃ¤hrt Reuter, daÃŸ es bei dem gegenwÃ


In [56]:
# split the text into words by space
words = text.split()

In [57]:
# print the 100th up the 120th words
words[100:120]

['respektooller',
 'Entfernung,',
 'Die',
 'beschrÃ¤nkten',
 'sich',
 'in',
 'der',
 'Hauptsache',
 'de',
 'Ortschaften',
 'im',
 'deutschen',
 'Hinter-',
 '<->',
 'mit',
 'Bombengeschwadern',
 'anzugreisen.',
 'der',
 'Zwischenzeit',
 'wurde']

In [58]:
# print the length of the word list
len(words)

11612

In [59]:
# Count the words with Counter and save the result to a variable
word_frequencies = Counter(words)

In [60]:
# 🚀 get the number of the word "Grippe" in the word frequencies 
word_frequencies["Grippe"]

4

In [61]:
# Ensure Bokeh output is displayed in the notebook
output_notebook()

# Convert the dictionary to a JSON string to be passed to javascript
word_freq_json = json.dumps(word_frequencies)

# Create the text input widget
text_input = TextInput(value='', title="Geben Sie ein Wort ein:")

# Create a Div to display the frequency
frequency_display = Div(text="Häufigkeit: ")

# JavaScript callback to update the frequency display
# Only needed for graphical interface 
callback = CustomJS(args=dict(frequency_display=frequency_display, text_input=text_input), code=f"""
    var word = text_input.value.trim();

    // Parse the word frequency dictionary from Python
    var word_freq = {word_freq_json};

    var frequency = word in word_freq ? word_freq[word] : "Nicht gefunden";
    frequency_display.text = "Häufigkeit: " + frequency;
""")

text_input.js_on_change('value', callback)

# Layout and display
layout = column(text_input, frequency_display)
show(layout)

Loading BokehJS ...

In [62]:
nlp = spacy.load('de_core_news_sm')

In [63]:
disable_components = ['ner', 'morphologizer', 'attribute_ruler', 'sentencizer']

In [64]:
# get the current time to display how long the annotation took
current = time()

# annotate with spacy
doc = nlp(text)

# extract tokens and lemmata, save them to a dictionary
text_annotated = {}
text_annotated['Token'] = [tok.text for tok in doc]
text_annotated['Lemma'] = [tok.lemma_ for tok in doc]

# convert the dictionary to a dataframe 
text_annotated_df = pd.DataFrame(text_annotated)

# calculate how long the annotation and extraction took and print result
took = time() - current
print(f"Die Annotation hat {round(took, 2)} Sekunden gedauert.") 

Die Annotation hat 2.77 Sekunden gedauert.


In [65]:
# print first five lines of the annotation
text_annotated_df.head()

,Token,Lemma
0,BERLINER,BERLINER
1,,
2,R,R
3,M,M
4,AHIRE,AHIRE


In [66]:
# get the lemmata 
text_tokenized = text_annotated_df.Lemma

# print the length
len(text_tokenized)

14981

In [67]:
# Count the words with Counter and save the result to a variable
token_frequencies = Counter(text_tokenized)

In [68]:
# 🚀 get the number of the word "Grippe" in the word frequencies 
token_frequencies["Grippe"]

5

In [69]:
# Ensure Bokeh output is displayed in the notebook
output_notebook()

# Convert the dictionary to a JSON string
tok_freq_json = json.dumps(token_frequencies)

# Create the text input widget
token_input = TextInput(value='', title="Geben Sie ein Wort ein:")

# Create a Div to display the frequency
token_frequency_display = Div(text="Häufigkeit: ")

# JavaScript callback to update the frequency display
# Only needed for graphical interface 
tok_callback = CustomJS(args=dict(frequency_display=token_frequency_display, text_input=token_input), code=f"""
    var tok = text_input.value.trim();

    // Parse the word frequency dictionary from Python
    var word_freq = {tok_freq_json};

    var frequency = tok in word_freq ? word_freq[tok] : "Nicht gefunden";
    frequency_display.text = "Häufigkeit: " + frequency;
""")

token_input.js_on_change('value', tok_callback)

# Layout and display
layout = column(token_input, token_frequency_display)
show(layout)

Loading BokehJS ...

In [70]:
# set output path to current directory
output_path = Path.cwd() / text_path.with_suffix(".csv").name

In [71]:
# set output path, change file extension
output_path = Path(r"data/csv") / text_path.with_suffix(".csv").name

In [72]:
# save the annotation as csv
text_annotated_df.to_csv(output_path, index=False)

In [73]:
# Call the function to plot the frequencies 
plot_with_js(corpus_annotations_merged, search_terms=word_list)

NameError: name 'plot_with_js' is not defined